In [1]:
import sys
sys.path.append('../src')
from BlackScholesVectorized import BlackScholesVectorized
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, date

# Loading and Processing Data

In [2]:
ticker = yf.Ticker('AAPL')
expiry = ticker.options[2]
options_chain = ticker.option_chain(expiry)
calls = options_chain.calls
puts = options_chain.puts

In [3]:
calls['option_type'] = 'call'
puts['option_type'] = 'put'

In [4]:
aapl_options = pd.concat([calls, puts], axis=0)
market_prices = aapl_options['lastPrice'] # Storing the current market prices, which are necessary to calcualte the implied volatiliry

options = aapl_options[['contractSymbol', 'strike', 'option_type']].copy()

In [5]:
expiry_date = datetime.strptime(expiry, '%Y-%m-%d').date()
today = date.today()
T = (expiry_date - today).days / 365
T

0.0410958904109589

In [6]:
options['St'] = 212.44
options['T'] = T
options['sigma'] = 0.179
options['r'] = 0.04
options['q'] = 0.0048
options

,contractSymbol,strike,option_type,St,T,sigma,r,q
0,AAPL250718C00090000,90.0,call,212.44,0.041096,0.179,0.04,0.0048
1,AAPL250718C00095000,95.0,call,212.44,0.041096,0.179,0.04,0.0048
2,AAPL250718C00100000,100.0,call,212.44,0.041096,0.179,0.04,0.0048
3,AAPL250718C00105000,105.0,call,212.44,0.041096,0.179,0.04,0.0048
4,AAPL250718C00110000,110.0,call,212.44,0.041096,0.179,0.04,0.0048
...,...,...,...,...,...,...,...,...
55,AAPL250718P00320000,320.0,put,212.44,0.041096,0.179,0.04,0.0048
56,AAPL250718P00325000,325.0,put,212.44,0.041096,0.179,0.04,0.0048
57,AAPL250718P00370000,370.0,put,212.44,0.041096,0.179,0.04,0.0048
58,AAPL250718P00380000,380.0,put,212.44,0.041096,0.179,0.04,0.0048


# Creating the Object
The ```BlackScholesVectorized``` object has to get spesific parameters it can identify: **symbol, St, K, T, sigma, r, q, option_type**, While the order does not matter.  
To provide the object the reqirued parameters, we can use two methods:
- 1. Cahnge the dataframe columns names to the required names.
- 2. Apply the ```column_map``` parameter, which takes a dictionary which maps the columns into the reqiured names. 

In [7]:
bs_map = {
    'symbol': 'contractSymbol',
    'St': 'St',
    'K': 'strike',
    'T': 'T',
    'sigma': 'sigma',
    'r': 'r',
    'q': 'q',
    'option_type': 'option_type'
}

bs = BlackScholesVectorized(options, column_map=bs_map)
bs

BlackScholesVectorized(n=125, map={'symbol': 'contractSymbol', 'St': 'St', 'K': 'strike', 'T': 'T', 'sigma': 'sigma', 'r': 'r', 'q': 'q', 'option_type': 'option_type'})

# 📈 Evaluating the Option Price

To calculate the theoretical fair value of the options under the Black-Scholes model, simply call the `evaluate()` method. This method requires no input arguments, as it relies on the parameters provided during initialization (e.g., spot price, strike, time to maturity, volatility, etc.).  
we can get only the values in two formarts:   
**1.** ```symbols=False```: Deafult. Retrieve a Numpay ndarray.  
**2.** ```symbols=True```: Retrieve a dictionary.  

Once called, it returns the Black-Scholes price of the option — either a **Call** or **Put**, depending on the specified `option_type`.

In [8]:
# for getting a Numpy ndarray ()
prices_array = bs.evaluate()
prices_array

array([ 1.22545922e+002,  1.17554134e+002,  1.12562347e+002,
        1.07570559e+002,  1.02578772e+002,  9.75869840e+001,
        9.25951964e+001,  8.76034088e+001,  8.26116212e+001,
        7.76198337e+001,  7.26280461e+001,  6.76362585e+001,
        6.26444709e+001,  5.76526834e+001,  5.26608958e+001,
        4.76691082e+001,  4.26773207e+001,  3.76855331e+001,
        3.26937477e+001,  3.01978665e+001,  2.77020406e+001,
        2.52064604e+001,  2.27118111e+001,  2.02201910e+001,
        1.77370891e+001,  1.52748032e+001,  1.28569617e+001,
        1.05224471e+001,  8.32577425e+000,  6.33120332e+000,
        4.60040360e+000,  3.17731024e+000,  2.07632407e+000,
        1.27903865e+000,  7.40591382e-001,  4.02239206e-001,
        2.04652632e-001,  4.34426346e-002,  7.08162024e-003,
        8.90895964e-004,  8.72059874e-005,  6.70830083e-006,
        4.09950103e-007,  2.01254414e-008,  8.02577235e-010,
        2.62834429e-011,  7.14333157e-013,  1.62751103e-014,
        3.13856195e-016,

In [9]:
# for a dictionary:
prices_dict = bs.evaluate(symbols=True)
prices_dict

{'AAPL250718C00090000': 122.54592183359985,
 'AAPL250718C00095000': 117.55413425989333,
 'AAPL250718C00100000': 112.5623466861868,
 'AAPL250718C00105000': 107.57055911248027,
 'AAPL250718C00110000': 102.57877153877375,
 'AAPL250718C00115000': 97.58698396506722,
 'AAPL250718C00120000': 92.5951963913607,
 'AAPL250718C00125000': 87.60340881765417,
 'AAPL250718C00130000': 82.61162124394764,
 'AAPL250718C00135000': 77.61983367024112,
 'AAPL250718C00140000': 72.6280460965346,
 'AAPL250718C00145000': 67.63625852282806,
 'AAPL250718C00150000': 62.64447094912154,
 'AAPL250718C00155000': 57.65268337541502,
 'AAPL250718C00160000': 52.66089580170848,
 'AAPL250718C00165000': 47.669108228002415,
 'AAPL250718C00170000': 42.67732065448084,
 'AAPL250718C00175000': 37.68553311076403,
 'AAPL250718C00180000': 32.6937477424226,
 'AAPL250718C00182500': 30.197866472213093,
 'AAPL250718C00185000': 27.70204055297424,
 'AAPL250718C00187500': 25.20646035670154,
 'AAPL250718C00190000': 22.711811075594483,
 'AAPL2

## ⚙️ `d1` and `d2` Parameters

The `d1` and `d2` properties return a dictionary of the intermediate terms used in the Black-Scholes formula. These values are essential components in the computation of the options price and Greeks.

- `d1` reflects the standardized distance between the current price and the strike price, adjusted for volatility and time.
- `d2` is derived directly from `d1` and represents the standardized distance accounting for the uncertainty of the asset's return.

These properties are computed automatically based on the model’s internal parameters (spot price, strike, volatility, interest rate, time to maturity, and dividend yield) and do not require any external input.

In [10]:
d1, d2 = bs.d1, bs.d2

In [11]:
list(d1.items())[:10]

[('AAPL250718C00090000', 23.610182097886245),
 ('AAPL250718C00095000', 22.120197353656057),
 ('AAPL250718C00100000', 20.70665649982315),
 ('AAPL250718C00105000', 19.362096920599758),
 ('AAPL250718C00110000', 18.080098105627425),
 ('AAPL250718C00115000', 16.855096218558057),
 ('AAPL250718C00120000', 15.682238159377803),
 ('AAPL250718C00125000', 14.55726543773236),
 ('AAPL250718C00130000', 13.476420835712009),
 ('AAPL250718C00135000', 12.43637269535011)]

In [12]:
list(d2.items())[:10]

[('AAPL250718C00090000', 23.57389500066619),
 ('AAPL250718C00095000', 22.083910256436003),
 ('AAPL250718C00100000', 20.670369402603097),
 ('AAPL250718C00105000', 19.325809823379704),
 ('AAPL250718C00110000', 18.04381100840737),
 ('AAPL250718C00115000', 16.818809121338003),
 ('AAPL250718C00120000', 15.645951062157751),
 ('AAPL250718C00125000', 14.520978340512308),
 ('AAPL250718C00130000', 13.440133738491957),
 ('AAPL250718C00135000', 12.400085598130058)]

## 🧮 Greeks

The `BlackScholesVectorized` class provides built-in support for calculating five key sensitivity measures - commonly referred to as the *Greeks* - which describe how the option price responds to changes in market parameters:

- **Delta (Δ):** Measures the sensitivity of the option price to changes in the underlying asset's price.
- **Gamma (Γ):** Measures the rate of change of Delta with respect to changes in the underlying price (i.e., the curvature).
- **Vega (ν):** Measures sensitivity to changes in the implied volatility of the underlying asset.
- **Theta (Θ):** Measures the rate of decline in the option’s value due to the passage of time (time decay).
- **Rho (ρ):** Measures sensitivity to changes in the risk-free interest rate.

Each of these Greeks is available as a dedicated method and is calculated analytically using the closed-form formulas derived from the Black-Scholes model.

These metrics are essential for risk management, hedging strategies, and understanding the behavior of options under different market scenarios.

We can get the values ain two formats:
1. ```symbols=False```: Deafult. Returns Numapy ndarrays
2. ```symbols=True```: Returns a dictionary.

In [13]:
deltas = bs.compute_delta()
deltas

array([ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        9.99999999e-01,  9.99999937e-01,  9.99996735e-01,  9.99981728e-01,
        9.99912763e-01,  9.99642102e-01,  9.98729618e-01,  9.96072078e-01,
        9.89349642e-01,  9.74504131e-01,  9.45742441e-01,  8.96626917e-01,
        8.22367204e-01,  7.22533218e-01,  6.02700620e-01,  4.73772325e-01,
        3.48967323e-01,  2.39875476e-01,  1.53472053e-01,  9.12579550e-02,
        5.04026201e-02,  1.23309742e-02,  2.26045264e-03,  3.13664159e-04,
        3.33526304e-05,  2.75344848e-06,  1.78834723e-07,  9.25754915e-09,
        3.86773441e-10,  1.31990133e-11,  3.72133634e-13,  8.76219482e-15,
        1.74062886e-16,  2.94543011e-18,  4.28419006e-20,  5.40212930e-22,
        5.95277753e-24,  

In [14]:
vegas = bs.compute_vega(symbols=True)
list(vegas.items())[:10]

[('AAPL250718C00090000', 1.5428931113586987e-122),
 ('AAPL250718C00095000', 9.643188981545038e-108),
 ('AAPL250718C00100000', 1.348259283396847e-94),
 ('AAPL250718C00105000', 6.738190636169787e-83),
 ('AAPL250718C00110000', 1.7856459760984263e-72),
 ('AAPL250718C00115000', 3.505572658046398e-63),
 ('AAPL250718C00120000', 6.78361788555506e-55),
 ('AAPL250718C00125000', 1.6539033462752154e-47),
 ('AAPL250718C00130000', 6.281801750459764e-41),
 ('AAPL250718C00135000', 4.4701530440774015e-35)]

## Summary
## 📊 Summary

To retrieve all the key outputs of the model at once — including the option price and its associated Greeks — use the `summary()` method.

This method returns a structured dictionary containing:
- The theoretical **option price**
- All five major **Greeks**: Delta, Gamma, Vega, Theta, and Rho

By default, the method returns a standard Pandas DataFrame. If you prefer a dictionary format, set the `as_frame=False` parameter to receive the results as a Python dictionary. A Pandas DataFrame makes it easier to visualize or export.

> ℹ️ There is no need to call `evaluate()` or `calculate_greek()` beforehand — `summary()` performs all required calculations internally.

In [15]:
summary = bs.summary()
summary

,AAPL250718C00090000,AAPL250718C00095000,AAPL250718C00100000,AAPL250718C00105000,AAPL250718C00110000,AAPL250718C00115000,AAPL250718C00120000,AAPL250718C00125000,AAPL250718C00130000,AAPL250718C00135000,...,AAPL250718P00290000,AAPL250718P00295000,AAPL250718P00300000,AAPL250718P00305000,AAPL250718P00315000,AAPL250718P00320000,AAPL250718P00325000,AAPL250718P00370000,AAPL250718P00380000,AAPL250718P00390000
price,1.225459e+02,1.175541e+02,1.125623e+02,1.075706e+02,1.025788e+02,9.758698e+01,9.259520e+01,8.760341e+01,8.261162e+01,7.761983e+01,...,7.712558e+01,8.211737e+01,8.710916e+01,9.210094e+01,1.020845e+02,1.070763e+02,1.120681e+02,1.569942e+02,1.669778e+02,1.769613e+02
delta,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
vega,1.542893e-122,9.643189e-108,1.348259e-94,6.738191e-83,1.785646e-72,3.505573e-63,6.783618e-55,1.653903e-47,6.281802e-41,4.470153e-35,...,1.109610e-17,1.699839e-19,2.250002e-21,2.594978e-23,2.355038e-27,1.879829e-29,1.345422e-31,1.205122e-52,1.161321e-57,8.988120e-63
gamma,4.647419e-123,2.904669e-108,4.061154e-95,2.029641e-83,5.378626e-73,1.055930e-63,2.043325e-55,4.981798e-48,1.892170e-41,1.346475e-35,...,3.342307e-18,5.120164e-20,6.777333e-22,7.816452e-24,7.093716e-28,5.662319e-30,4.052607e-32,3.630004e-53,3.498068e-58,2.707353e-63
theta,-9.846814e-03,-1.039386e-02,-1.094090e-02,-1.148795e-02,-1.203499e-02,-1.258204e-02,-1.312909e-02,-1.367613e-02,-1.422318e-02,-1.477022e-02,...,3.172862e-02,3.227567e-02,3.282271e-02,3.336976e-02,3.446385e-02,3.501089e-02,3.555794e-02,4.048135e-02,4.157544e-02,4.266953e-02
rho,3.692555e-02,3.897697e-02,4.102839e-02,4.307981e-02,4.513123e-02,4.718265e-02,4.923407e-02,5.128549e-02,5.333691e-02,5.538833e-02,...,-1.189823e-01,-1.210338e-01,-1.230852e-01,-1.251366e-01,-1.292394e-01,-1.312909e-01,-1.333423e-01,-1.518050e-01,-1.559079e-01,-1.600107e-01


### 📈 Sensitivity Analysis

The `BlackScholes` class includes three specialized methods designed to evaluate how the **option price** and its **Greeks** respond when a single parameter is varied, while all others remain constant:

- `evaluate_over_time(T_range)`  
Varies the **time to maturity (T)** over a given range. Useful for understanding how time decay affects option prices.

- `greeks_over_time(T_range)`  
Varies **time to maturity**. Useful for understanding how time decay affects the options greeks.

- `greeks_over_price(St_range)`  
Varies the **underlying asset price**. Useful for understanding how the underlying asset price affects the options greeks.

- `greeks_over_volatility(sigma_range)`  
  Varies **implied volatility**. Useful for understanding how the underlying asset volatility affects the options greeks.

#### Input:
Each method accepts three argument:
1. A **range of strictly positive values** (list, NumPy array, or similar) corresponding to the parameter being varied.
2. ```as_frame```: Optional, Deafult is ```True```.
3. ```prices```: Optional. Deafult is ```False```. 

#### Output:
All three methods return a **Pandas DataFrame** where:
- Each **row** corresponds to one value from the input range  
- Each **column** contains:
  - The **option price**
  - The five primary **Greeks**: Delta, Gamma, Vega, Theta, and Rho

This format is ideal for further analysis, visualization, or exporting to external tools.

> ⚠️ **Important:** All input values must be strictly positive. Negative or zero values are not valid in the Black-Scholes model and will raise errors or produce invalid results.

### 1. Evaluate Over Time

In [16]:
trange = np.linspace(0.01, 1, 100)
price_over_time = bs.evaluate_over_time(trange, 1)

In [17]:
price_over_time.head()

,AAPL250718C00090000,AAPL250718C00095000,AAPL250718C00100000,AAPL250718C00105000,AAPL250718C00110000,AAPL250718C00115000,AAPL250718C00120000,AAPL250718C00125000,AAPL250718C00130000,AAPL250718C00135000,...,AAPL250718P00290000,AAPL250718P00295000,AAPL250718P00300000,AAPL250718P00305000,AAPL250718P00315000,AAPL250718P00320000,AAPL250718P00325000,AAPL250718P00370000,AAPL250718P00380000,AAPL250718P00390000
0.01,122.465796,117.467796,112.469795,107.471795,102.473794,97.475794,92.477794,87.479793,82.481793,77.483792,...,77.454220,82.452220,87.450221,92.448221,102.444222,107.442222,112.440223,157.422226,167.418227,177.414228
0.02,122.491578,117.495576,112.499575,107.503573,102.507572,97.511570,92.515568,87.519567,82.523565,77.527564,...,77.348486,82.344488,87.340489,92.336491,102.328494,107.324496,112.320497,157.284512,167.276515,177.268518
0.03,122.517346,117.523342,112.529339,107.535335,102.541332,97.547328,92.553324,87.559321,82.565317,77.571314,...,77.242798,82.236801,87.230805,92.224809,102.212816,107.206819,112.200823,157.146855,167.134863,177.122870
0.04,122.543100,117.551094,112.559088,107.567081,102.575075,97.583068,92.591062,87.599056,82.607049,77.615043,...,77.137156,82.129162,87.121168,92.113175,102.097188,107.089194,112.081200,157.009258,166.993271,176.977283
0.05,122.568841,117.578831,112.588821,107.598811,102.608801,97.618791,92.628781,87.638771,82.648761,77.658751,...,77.031559,82.021569,87.011579,92.001589,101.981609,106.971619,111.961629,156.871719,166.851739,176.831759


### 2. Greeks Over Time

In [18]:
got = bs.greeks_over_time(trange)
got

AAPL250718C00090000  AAPL250718C00095000  AAPL250718C00100000  \
time greek                                                                  
0.01 delta         1.000000e+00             1.000000             1.000000   
     vega          0.000000e+00             0.000000             0.000000   
     gamma         0.000000e+00             0.000000             0.000000   
     theta        -9.859069e-03            -0.010407            -0.010955   
     rho           8.996401e-03             0.009496             0.009996   
...                         ...                  ...                  ...   
1.00 delta         9.999968e-01             0.999987             0.999956   
     vega          3.218362e-05             0.000120             0.000385   
     gamma         3.983908e-07             0.000001             0.000005   
     theta        -9.476999e-03            -0.010005            -0.010538   
     rho           8.647041e-01             0.912725             0.960703   

            AAPL250718C00105000  AAPL250718C00110000  AAPL250718C00115000  \
time greek                                                                  
0.01 delta             1.000000         1.000000e+00         1.000000e+00   
     vega              0.000000        6.281532e-295        1.214206e-256   
     gamma             0.000000        7.775708e-295        1.503027e-256   
     theta            -0.011502        -1.204997e-02        -1.259770e-02   
     rho               0.010496         1.099560e-02         1.149540e-02   
...                         ...                  ...                  ...   
1.00 delta             0.999869         9.996516e-01         9.991624e-01   
     vega              0.001082         2.700506e-03         6.077919e-03   
     gamma             0.000013         3.342870e-05         7.523662e-05   
     theta            -0.011079        -1.164070e-02        -1.223920e-02   
     rho               1.008568         1.056171e+00         1.103227e+00   

            AAPL250718C00120000  AAPL250718C00125000  AAPL250718C00130000  \
time greek                                                                  
0.01 delta         1.000000e+00         1.000000e+00         1.000000e+00   
     vega         1.702605e-222        4.753108e-192        6.324052e-165   
     gamma        2.107601e-222        5.883721e-192        7.828343e-165   
     theta        -1.314543e-02        -1.369315e-02        -1.424088e-02   
     rho           1.199520e-02         1.249500e-02         1.299480e-02   
...                         ...                  ...                  ...   
1.00 delta         9.981622e-01         9.962822e-01         9.930059e-01   
     vega          1.247301e-02         2.357068e-02         4.136712e-02   
     gamma         1.543994e-04         2.917740e-04         5.120704e-04   
     theta        -1.290035e-02        -1.365720e-02        -1.454699e-02   
     rho           1.149249e+00         1.193480e+00         1.234854e+00   

            AAPL250718C00135000  ...  AAPL250718P00290000  \
time greek                       ...                        
0.01 delta         1.000000e+00  ...        -1.000000e+00   
     vega         8.565752e-141  ...         1.172524e-67   
     gamma        1.060327e-140  ...         1.451430e-67   
     theta        -1.478860e-02  ...         3.176811e-02   
     rho           1.349460e-02  ...        -2.898840e-02   
...                         ...  ...                  ...   
1.00 delta         9.876713e-01  ...        -9.785568e-01   
     vega          6.792558e-02  ...         1.091113e-01   
     gamma         8.408291e-04  ...         1.350655e-03   
     theta        -1.560519e-02  ...         2.743879e-02   
     rho           1.271990e+00  ...        -2.747926e+00   

            AAPL250718P00295000  AAPL250718P00300000  AAPL250718P00305000  \
time greek                                                                  
0.01 delta        -1.000000e+00        -1.000000e+00      

### 3. Greeks Over Price

In [19]:
srange = np.linspace(190, 220, 100)
gos = bs.greeks_over_price(srange)
gos

AAPL250718C00090000  AAPL250718C00095000  AAPL250718C00100000  \
St    greek                                                                  
190.0 delta         1.000000e+00         1.000000e+00         1.000000e+00   
      vega          4.264716e-93         2.722869e-80         4.920022e-69   
      gamma         1.605948e-93         1.025341e-80         1.852714e-69   
      theta        -9.846814e-03        -1.039386e-02        -1.094090e-02   
      rho           3.692555e-02         3.897697e-02         4.102839e-02   
...                          ...                  ...                  ...   
220.0 delta         1.000000e+00         1.000000e+00         1.000000e+00   
      vega         1.320870e-132        3.469870e-117        1.894271e-103   
      gamma        3.709908e-133        9.745775e-118        5.320412e-104   
      theta        -9.846814e-03        -1.039386e-02        -1.094090e-02   
      rho           3.692555e-02         3.897697e-02         4.102839e-02   

             AAPL250718C00105000  AAPL250718C00110000  AAPL250718C00115000  \
St    greek                                                                  
190.0 delta         1.000000e+00         1.000000e+00         1.000000e+00   
      vega          3.929066e-59         2.016882e-50         9.139782e-43   
      gamma         1.479554e-59         7.594895e-51         3.441733e-43   
      theta        -1.148795e-02        -1.203499e-02        -1.258204e-02   
      rho           4.307981e-02         4.513123e-02         4.718265e-02   
...                          ...                  ...                  ...   
220.0 delta         1.000000e+00         1.000000e+00         1.000000e+00   
      vega          3.458748e-91         3.152796e-80         2.015259e-70   
      gamma         9.714537e-92         8.855215e-81         5.660229e-71   
      theta        -1.148795e-02        -1.203499e-02        -1.258204e-02   
      rho           4.307981e-02         4.513123e-02         4.718265e-02   

             AAPL250718C00120000  AAPL250718C00125000  AAPL250718C00130000  \
St    greek                                                                  
190.0 delta         1.000000e+00         1.000000e+00         1.000000e+00   
      vega          4.792904e-36         3.669331e-30         5.012557e-25   
      gamma         1.804846e-36         1.381746e-30         1.887560e-25   
      theta        -1.312909e-02        -1.367613e-02        -1.422318e-02   
      rho           4.923407e-02         5.128549e-02         5.333691e-02   
...                          ...                  ...                  ...   
220.0 delta         1.000000e+00         1.000000e+00         1.000000e+00   
      vega          1.207486e-61         8.704407e-54         9.368151e-47   
      gamma         3.391448e-62         2.444795e-54         2.631219e-47   
      theta        -1.312909e-02        -1.367613e-02        -1.422318e-02   
      rho           4.923407e-02         5.128549e-02         5.333691e-02   

             AAPL250718C00135000  ...  AAPL250718P00290000  \
St    greek                       ...                        
190.0 delta         1.000000e+00  ...        -1.000000e+00   
      vega          1.454466e-20  ...         2.540104e-31   
      gamma         5.477027e-21  ...         9.565173e-32   
      theta        -1.477022e-02  ...         3.172862e-02   
      rho           5.538833e-02  ...        -1.189823e-01   
...                          ...  ...                  ...   
220.0 delta         1.000000e+00  ...        -1.000000e+00   
      vega          1.816180e-40  ...         2.967337e-14   
      gamma         5.101081e-41  ...         8.334317e-15   
      theta        -1.477022e-02  ...         3.172862e-02   
      rho           5.538833e-02  ...        -1.189823e-01   

             AAPL250718P00295000  AAPL250718P00300000  AAPL250718P00305000  \
St    greek                                                                  
190.0

# 4. Greeks Over Volatility

In [20]:
vrange = np.linspace(0.001, 1, 100)
gov = bs.greeks_over_volatility(vrange)
gov

AAPL250718C00090000  AAPL250718C00095000  AAPL250718C00100000  \
time  greek                                                                  
0.001 delta             1.000000             1.000000             1.000000   
      vega              0.000000             0.000000             0.000000   
      gamma             0.000000             0.000000             0.000000   
      theta            -0.009847            -0.010394            -0.010941   
      rho               0.036926             0.038977             0.041028   
...                          ...                  ...                  ...   
1.000 delta             0.999982             0.999944             0.999846   
      vega              0.000034             0.000099             0.000256   
      gamma             0.000002             0.000005             0.000014   
      theta            -0.009961            -0.010724            -0.011790   
      rho               0.036924             0.038972             0.041015   

             AAPL250718C00105000  AAPL250718C00110000  AAPL250718C00115000  \
time  greek                                                                  
0.001 delta             1.000000             1.000000             1.000000   
      vega              0.000000             0.000000             0.000000   
      gamma             0.000000             0.000000             0.000000   
      theta            -0.011488            -0.012035            -0.012582   
      rho               0.043080             0.045131             0.047183   
...                          ...                  ...                  ...   
1.000 delta             0.999621             0.999150             0.998244   
      vega              0.000592             0.001249             0.002426   
      gamma             0.000032             0.000067             0.000131   
      theta            -0.013452            -0.016178            -0.020627   
      rho               0.043046             0.045056             0.047027   

             AAPL250718C00120000  AAPL250718C00125000  AAPL250718C00130000  \
time  greek                                                                  
0.001 delta             1.000000             1.000000             1.000000   
      vega              0.000000             0.000000             0.000000   
      gamma             0.000000             0.000000             0.000000   
      theta            -0.013129            -0.013676            -0.014223   
      rho               0.049234             0.051285             0.053337   
...                          ...                  ...                  ...   
1.000 delta             0.996626             0.993923             0.989671   
      vega              0.004380             0.007406             0.011807   
      gamma             0.000236             0.000399             0.000637   
      theta            -0.027649            -0.038217            -0.053333   
      rho               0.048934             0.050743             0.052411   

             AAPL250718C00135000  ...  AAPL250718P00290000  \
time  greek                       ...                        
0.001 delta             1.000000  ...            -1.000000   
      vega              0.000000  ...             0.000000   
      gamma             0.000000  ...             0.000000   
      theta            -0.014770  ...             0.031729   
      rho               0.055388  ...            -0.118982   
...                          ...  ...                  ...   
1.000 delta             0.983331  ...            -0.949882   
      vega              0.017853  ...             0.044500   
      gamma             0.000963  ...             0.002399   
      theta            -0.073881  ...            -0.117632   
      rho               0.053887  ...            -0.115126   

             AAPL250718P00295000  AAPL250718P00300000  AAPL250718P00305000  \
time  greek                                                                  
0.001

### 📌 Implied Volatility

The `implied_volatility()` method allows you to compute the **implied volatility** of an option based on its current **market price**.

Implied volatility is the level of volatility that, when plugged into the Black-Scholes formula, yields the given market price. It is a reverse-engineering process, often used by traders to infer market expectations of future volatility.

#### Usage:
To use this method, you must supply the **current market price** of the option as an argument

#### Parameters:
- `market_price` (`float`): The observed price of the option in the market. Must be a positive value.
- `n_jobs` (`int`): The number of CPU cores we can to use (deafult is `1`).

#### Returns:
- A single **float** representing the implied volatility (annualized), typically expressed in decimal form (e.g., `0.24` for 24%).

> ⚠️ This method uses numerical root-finding (e.g., Brent’s method) under the hood. It may fail or return `np.nan` if the market price is not within a plausible range given the model’s assumptions.

In [21]:
iv = bs.implied_volatility(np.array(market_prices), n_jobs=4)

In [22]:
iv

array([       nan,        nan,        nan,        nan,        nan,
       1.48241375,        nan,        nan,        nan,        nan,
              nan,        nan,        nan, 0.84769682,        nan,
       0.68806019, 0.60360951, 0.57638061, 0.47700605,        nan,
       0.46519127,        nan, 0.3796562 ,        nan, 0.33440424,
       0.25168361, 0.29489355, 0.28534406, 0.27708283, 0.26712658,
       0.26138333, 0.2546147 , 0.24815361, 0.24467352, 0.24065363,
       0.23952024, 0.23922585, 0.25127745, 0.27512765, 0.29658464,
       0.32869046, 0.35575046, 0.36655144, 0.40072797, 0.42023923,
       0.43310305, 0.4627711 , 0.46089248, 0.48770008, 0.51391904,
       0.53958031, 0.56471188, 0.58933924, 0.65260633, 0.63717267,
       0.66041996, 0.68324589, 0.70566748, 0.74936014, 0.79161306,
       0.88315752, 0.87219401, 0.91069257, 0.94809571, 0.98446876,
              nan, 1.49843887, 1.33940834, 1.22501295, 1.12948524,
       1.04505705, 0.9682276 , 0.89707295, 0.88659487, 0.81929